In [1]:
from osgeo import gdal,ogr
gdal.UseExceptions()
import numpy as np
import matplotlib.pyplot as plt

# Parabolic Bowl Topography

In [ ]:
X_LENGTH = 40
MAX_DEPTH = -1000
Y_LENGTH = 11
X_RES = 0.1
Y_RES = 0.1
H_0 = 5
a = 10

x_coord = np.linspace(0, X_LENGTH,  int(X_LENGTH / X_RES)+1)
y_coord = np.linspace(0, Y_LENGTH,  int(Y_LENGTH / Y_RES)+1)
xx, yy = np.meshgrid(x_coord,y_coord)
height = (H_0) *( (1/a) * (1/a) * np.square(xx - (X_LENGTH / 2)) - 1) 
plt.imshow(height)
plt.colorbar()

In [ ]:
#Output to geospatial File
FILENAME = 'parabola.tif'
driver = gdal.GetDriverByName('GTiff')
GDT_dtype = gdal.GDT_Float32
outRaster = driver.Create(FILENAME, int(X_LENGTH / X_RES)+1,int(Y_LENGTH / Y_RES)+1, 1, GDT_dtype)
outRaster.SetGeoTransform((0, X_RES, 0, 0, 0, Y_RES))
outband = outRaster.GetRasterBand(1)
outband.WriteArray(height)
outband.FlushCache()

# Tsunami ISEC Beach Topography

In [ ]:
# Long Beach
X_LENGTH = 50500
Y_LENGTH = 20
X_RES = 1
Y_RES = 1
SLOPE = 0.1
FILENAME = 'isec-slope.tif'

base = np.linspace(0,X_LENGTH*SLOPE, int(X_LENGTH / X_RES)+1)
base = np.vstack(base)
base = np.repeat(base,int(Y_LENGTH/Y_RES)+1,axis=1).transpose()
driver = gdal.GetDriverByName('GTiff')
GDT_dtype = gdal.GDT_Float32
outRaster = driver.Create(FILENAME, int(X_LENGTH / X_RES)+1,int(Y_LENGTH / Y_RES)+1, 1, GDT_dtype)
outRaster.SetGeoTransform((0, X_RES, 0, 0, 0, Y_RES))
outband = outRaster.GetRasterBand(1)
outband.WriteArray(base)
outband.FlushCache()

## Frictioned Beach Topography

In [2]:
# Long Beach
X_LENGTH = 1000
Y_LENGTH = 200
X_RES = 1
Y_RES = 1
SLOPE = 0.05
FILENAME = 'frictioned_benchmark.tif'

base = np.linspace(0,X_LENGTH*SLOPE, int(X_LENGTH / X_RES)+1)
base = np.vstack(base)
base = np.repeat(base,int(Y_LENGTH/Y_RES)+1,axis=1).transpose()
driver = gdal.GetDriverByName('GTiff')
GDT_dtype = gdal.GDT_Float32
outRaster = driver.Create(FILENAME, int(X_LENGTH / X_RES)+1,int(Y_LENGTH / Y_RES)+1, 1, GDT_dtype)
outRaster.SetGeoTransform((0, X_RES, 0, 0, 0, Y_RES))
outband = outRaster.GetRasterBand(1)
outband.WriteArray(base)
outband.FlushCache()

# Hill Mitigation Topography

In [ ]:
# Underlying Beach
X_LENGTH = 12000
MAX_DEPTH = -1000
Y_LENGTH = 220
X_RES = 1
Y_RES = 1
SLOPE = 1/50

# Hill Configuration
HILL_H = 15
FIRST_HILL_CENTER_X = 4000
FIRST_HILL_CENTER_Y = 50
HILL_RADIUS = 30
HILL_COUNT_X = 1
HILL_COUNT_Y = 2
SPACING_X = 70
SPACING_Y = 120
base = np.linspace(-X_LENGTH*SLOPE/2,X_LENGTH*SLOPE/2, int(X_LENGTH / X_RES)+1)
base[base<0] = 0
base = np.vstack(base)
base = np.repeat(base,int(Y_LENGTH/Y_RES)+1,axis=1).transpose()
x_coord = np.linspace(0, X_LENGTH,  int(X_LENGTH / X_RES)+1)
y_coord = np.linspace(0, Y_LENGTH,  int(Y_LENGTH / Y_RES)+1)
xx, yy = np.meshgrid(x_coord,y_coord)
l = np.repeat(xx[:, :, np.newaxis], HILL_COUNT_X*HILL_COUNT_Y, axis=2)
for i in range(HILL_COUNT_X):
    for j in range(HILL_COUNT_Y):
        l[:,:,i*HILL_COUNT_X +j] = np.sqrt(np.square(xx - (FIRST_HILL_CENTER_X + i*SPACING_X))+np.square(yy - (FIRST_HILL_CENTER_Y + j*SPACING_Y)))
l = np.min(l, axis = 2)
ellipse = lambda x: x*HILL_H/HILL_RADIUS*np.tan(np.arccos(x/HILL_RADIUS))
cone = lambda x: HILL_H/HILL_RADIUS*(HILL_RADIUS-x)
# choose either conical or ellipsoidal hills
hills = np.piecewise(l, [l > HILL_RADIUS, l<=HILL_RADIUS], [0, ellipse])
result = base + hills
plt.imshow(result)
plt.colorbar()

In [ ]:
#Output to geospatial File
FILENAME = 'double-hill-configuration.tif'
driver = gdal.GetDriverByName('GTiff')
GDT_dtype = gdal.GDT_Float32
outRaster = driver.Create(FILENAME, int(X_LENGTH / X_RES)+1,int(Y_LENGTH / Y_RES)+1, 1, GDT_dtype)
outRaster.SetGeoTransform((0, X_RES, 0, 0, 0, Y_RES))
outband = outRaster.GetRasterBand(1)
outband.WriteArray(result)
outband.FlushCache()